# Figures recognition - Functional Demo

In this notebook, we will link some certain actions to the recognition of figures.

Importing libraries and the model

In [ ]:
import torch
import torchvision
from jetbot import Robot

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 5)
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
model = model.to(device)
model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

robot = Robot()

<br/>

Preprocessing of the data

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

<br/>

Creating widgets for the "recognition level" sliders

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
arrow_slider = widgets.FloatSlider(description='arrow', min=0.0, max=1.0, orientation='vertical')
square_slider = widgets.FloatSlider(description='square', min=0.0, max=1.0, orientation='vertical')
circle_slider = widgets.FloatSlider(description='circle', min=0.0, max=1.0, orientation='vertical')
free_slider = widgets.FloatSlider(description='free', min=0.0, max=1.0, orientation='vertical')
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

<br/>

### Logic of the robot

When our Jetbot recognises certain figures it performs some preprogrammed actions:
* Red circle - turn right
* Blue arrow - turn left
* Green square - stop
* Free space - go forward
* Blocked space - stop

In [ ]:
import torch.nn.functional as F
import time

current_direction = 0
max_index = 0

def update(change):
    global blocked_slider, robot, arrow_slider, square_slider, circle_slider, free_slider, current_direction, max_index
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    speedvalues = [0.1, 0.2, 0.3, 0.4]
    prob_blocked = float(y.flatten()[1])
    prob_arrow = float(y.flatten()[0])
    prob_square = float(y.flatten()[4])
    prob_circle = float(y.flatten()[2])
    prob_free = float(y.flatten()[3]) 
    blocked_slider.value = prob_blocked
    arrow_slider.value = prob_arrow
    square_slider.value = prob_square
    circle_slider.value = prob_circle
    free_slider.value = prob_free
    
    # we decided that instead of performing some action when recognition level of some figure exceeds predefined level
    # our robot will do something when some certain element has been recognised for a longer period of time
    # in that way we are improving stability and preventing unwanted actions
    
    dir_list = [prob_blocked, prob_arrow, prob_square, prob_circle, prob_free]
    max_value = max(dir_list)
    max_index_tmp = dir_list.index(max_value)
    
    if max_index_tmp != max_index:
        current_direction = 1
    else:
        current_direction += 1
    max_index = max_index_tmp
    if current_direction > 2:
        if max_index == 0:      #blocked_prob
            robot.stop()
        elif max_index == 1:    #arrow_prob
            robot.left(0.24)
        elif max_index == 2:    #square_prob
            robot.stop()
        elif max_index == 3:    #circle_prob
            robot.right(0.23)
        elif max_index == 4:    #free_prob
            robot.set_motors(0.29, 0.3)
    
    time.sleep(0.01) #how often we take new samples
        
update({'new': camera.value})

<br/>

Getting updates from the camera

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

<br/>

Stopping the robot

In [ ]:
import time

camera.unobserve(update, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

<br/>

If you would like to completely shut down the robot

In [ ]:
camera_link.unlink()
camera.stop()